In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v10.csv"
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v11.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_CHAOS_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_LVSC_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_2_20_2024_merged_v13.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_16_2023_merged_edited1.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_5_31_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_5_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_13_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_20_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_5_20_2024_lab_v15.csv"
results_csv_file = r"/Users/admin/Documents/research/active learning/results/results_8_1_2024_lab.csv"
#results_csv_file = r"/Users/admin/Documents/research/active learning/results/results_8_1_2024_lab_v15.csv"
#results_csv_file = r"/Users/admin/Documents/research/active learning/results/results_8_3_2024_lab_v18.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  
ignore_string_in_name_lst = []
include_string_in_name_lst = ["sup", "CHAOS"]

In [4]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 5
num_rounds = 10
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [8]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    continue_flag = False
    for ignore_string_in_name in ignore_string_in_name_lst:
        if ignore_string_in_name in base_exp_name:
            continue_flag = True
            break
    for include_string_in_name in include_string_in_name_lst:
        if include_string_in_name not in base_exp_name:
            print("rejected, ", base_exp_name)
            continue_flag = True
            break
    if continue_flag:
        continue
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print("accepted ", base_exp_name)

rejected,  DMPLS_MSCMR__coreset_fuse05_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_uncertainty1_v13
rejected,  DMPLS_MSCMR__coreset_fuse10_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_label2_uncertainty1_v15
rejected,  DMPLS_MSCMR__coreset_fuse10_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_uncertainty1_v13
rejected,  DMPLS_MSCMR__coreset_fuse20_pos_loss1_wt02_use_slice_pos_uncertainty1_v13
rejected,  DMPLS_MSCMR__coreset_v13
rejected,  DMPLS_MSCMR__stochastic_v13
rejected,  DMPLS_MSCMR__typiclust_v10
rejected,  DMPLS__bald_v10
rejected,  DMPLS__coregcn_k_slice_wt_v5
rejected,  DMPLS__coreset_cl_bs240_v5
rejected,  DMPLS__coreset_fuse01_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_label2_uncertainty1_v15
rejected,  DMPLS__coreset_fuse05_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_v15
rejected,  DMPLS__coreset_fuse10_loss_wt02_ext

In [9]:
index = pd.Index(list(results_dict.keys()), name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
mean_strings = round_strings[0:5]
mean_strings = round_strings[1:5]
#mean_strings = round_strings[0:4]
mean_string_col = "mean_test"
df = pd.DataFrame(index=index, 
                  columns=[mean_string_col] + sum(round_counts_strings, []))

In [10]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    mean_result_list = []
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results
            if round_string in mean_strings:
                mean_result_list.append(mean_result)
        sub_df[mean_string_col] = np.sum(mean_result_list)/len(mean_result_list)

/var/folders/d4/_mqx502j3f98n_z9jm_d2g900000gq/T/ipykernel_87031/2566401622.py:17: RuntimeWarning: invalid value encountered in scalar divide
  sub_df[mean_string_col] = np.sum(mean_result_list)/len(mean_result_list)
/opt/miniconda3/envs/neph_kg/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/miniconda3/envs/neph_kg/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
df
df.sort_values(by=[mean_string_col], ascending=False)

,mean_test,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count,round5_test,round5_count,round6_test,round6_count,round7_test,round7_count,round8_test,round8_count,round9_test,round9_count
exp_name,,,,,,,,,,,,,,,,,,,,,
DMPLS_sup_CHAOS__typiclust_v10,0.863668,NaN,NaN,0.816+/-nan,1,0.857+/-nan,1,0.890+/-nan,1,0.891+/-nan,1,0.937+/-nan,1,0.957+/-nan,1,0.956+/-nan,1,NaN,NaN,NaN,NaN
DMPLS_sup_CHAOS__coreset_pos_loss1_wt035_use_phase_v17,0.848086,0.726+/-0.075,5,0.805+/-0.082,5,0.825+/-0.060,5,0.859+/-0.057,5,0.903+/-0.033,5,0.925+/-0.022,5,0.944+/-0.013,5,0.956+/-0.008,5,0.963+/-0.004,5,NaN,NaN
DMPLS_sup_CHAOS__coregcn_v16,0.847417,0.747+/-0.093,3,0.777+/-0.119,3,0.807+/-0.137,3,0.906+/-0.025,3,0.900+/-0.009,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS_sup_CHAOS__coreset_pos_loss1_wt03_pos_loss2_wt01_use_slice_pos_use_phase_v17,0.842401,0.716+/-0.093,5,0.798+/-0.057,5,0.838+/-0.041,5,0.894+/-0.043,5,0.840+/-0.046,5,0.926+/-0.026,5,0.927+/-0.024,5,0.949+/-0.011,5,0.964+/-0.005,5,NaN,NaN
DMPLS_sup_CHAOS__coreset_v16,0.829612,0.774+/-0.074,3,0.883+/-0.024,3,0.816+/-0.045,3,0.754+/-0.025,3,0.866+/-0.054,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS_sup_CHAOS__random_v17,0.829126,0.785+/-0.061,5,0.807+/-0.046,5,0.817+/-0.042,5,0.842+/-0.049,5,0.851+/-0.070,5,0.927+/-0.014,5,0.943+/-0.014,5,0.948+/-0.007,5,0.962+/-0.005,5,NaN,NaN
DMPLS_sup_CHAOS__coreset_v17,0.824426,0.772+/-0.067,5,0.800+/-0.040,5,0.804+/-0.054,5,0.812+/-0.035,5,0.881+/-0.039,5,0.922+/-0.030,5,0.949+/-0.008,5,0.958+/-0.005,5,0.965+/-0.005,5,NaN,NaN
DMPLS_sup_CHAOS__stochastic_v17,0.821275,0.778+/-0.075,5,0.772+/-0.090,5,0.829+/-0.069,5,0.838+/-0.078,5,0.847+/-0.050,5,0.921+/-0.028,5,0.930+/-0.030,5,0.941+/-0.016,5,0.961+/-0.006,5,NaN,NaN
DMPLS_sup_CHAOS__coregcn_v17,0.81712,0.776+/-0.060,5,0.772+/-0.068,5,0.784+/-0.060,5,0.868+/-0.034,5,0.845+/-0.025,5,0.910+/-0.018,5,0.941+/-0.013,5,0.953+/-0.005,5,0.961+/-0.007,5,NaN,NaN
